# Football Data from Transfermarkt
- Kaggle Data URL

    - https://www.kaggle.com/datasets/davidcariboo/player-scores

## 데이터 설명
- Transfermarkt 웹사이트에서 스크레이핑한 각종 축구 데이터 (매주 한번 파일 업데이트)
    - 주요 대회 시즌별 60,000개 이상의 경기
    - 모든 대회의 400개 이상의 클럽들
    - 30,000 이상의 선수들
    - 400,000개 이상의 선수 가치 기록
    - 1,200,000개 이상의 선수 출전 기록

- `csv` 파일 설명
    - `apperances` : 선수 출장 기록
    - `club_games` : 클럽별 경기 홈팀, 어웨이팀 정보
    - `clubs` : 리그별 속해 있는 클럽
    - `competitions` : 대회 정보
    - `game_events` : 경기별 이벤트 정보 (카드, 득점, 어시스트 등)
    - `game_lineups` : 경기별 선수의 선발, 교체 명단 등재 여부
    - `games` : 경기에 대한 정보 (시즌, 라운드, 홈팀, 어웨이팀, 순위)
    - `player_valuations` : Transfermarkt 웹사이트에서 매긴 선수의 가치
    - `players` : 선수에 대한 세부 정보

- 데이터베이스 스키마
    <img src="https://raw.githubusercontent.com/dcaribou/transfermarkt-datasets/master/resources/diagram.svg?sanitize=true" width="1700">

In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from datetime import datetime
from datetime import timedelta
import missingno as msno

In [48]:
# 데이터 경로 지정
# C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data
apps_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/appearances.csv"
club_games_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/club_games.csv"
clubs_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/clubs.csv"
competitions_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/competitions.csv"
game_events_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/game_events.csv"
game_lineups_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/game_lineups.csv"
games_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/games.csv"
player_valuations_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/player_valuations.csv"
players_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/players.csv"

# 데이터셋 불러오기
apps_df = pd.read_csv(apps_path)
club_games_df = pd.read_csv(club_games_path)
clubs_df = pd.read_csv(clubs_path)
competitions_df = pd.read_csv(competitions_path)
game_events_df = pd.read_csv(game_events_path)
game_lineups_df = pd.read_csv(game_lineups_path)
games_df = pd.read_csv(games_path)
player_valuations_df = pd.read_csv(player_valuations_path)
players_df = pd.read_csv(players_path)

In [49]:
apps_df.head(3)

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
0,2231978_38004,2231978,38004,853,235,2012-07-03,Aurélien Joachim,CLQ,0,0,2,0,90
1,2233748_79232,2233748,79232,8841,2698,2012-07-05,Ruslan Abyshov,ELQ,0,0,0,0,90
2,2234413_42792,2234413,42792,6251,465,2012-07-05,Sander Puri,ELQ,0,0,0,0,45


In [50]:
club_games_df.head(3)

,game_id,club_id,own_goals,own_position,own_manager_name,opponent_id,opponent_goals,opponent_position,opponent_manager_name,hosting,is_win
0,2320450,1468,0,NaN,Holger Bachthaler,24,2,NaN,Armin Veh,Home,0
1,2320460,1,3,NaN,Jürgen Luginger,86,1,NaN,Robin Dutt,Home,1
2,2320472,2036,4,NaN,Frank Schmidt,72,5,NaN,Alexander Schmidt,Home,0


In [51]:
clubs_df.tail(3)

,club_id,club_code,name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url
423,800,atalanta-bergamo,Atalanta Bergamasca Calcio S.p.a.,IT1,NaN,25,26.6,17,68.0,16,Gewiss Stadium,21747,+€73.82m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/atalanta-berga...
424,979,moreirense-fc,Moreirense Futebol Clube,PO1,NaN,29,25.6,17,58.6,3,Estádio C. J. de Almeida Freitas,6153,€-2.05m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/moreirense-fc/...
425,984,west-bromwich-albion,West Bromwich Albion,GB1,NaN,25,27.8,8,32.0,4,The Hawthorns,26850,€-1.35m,NaN,2020,../data/raw/transfermarkt-scraper/2020/clubs.j...,https://www.transfermarkt.co.uk/west-bromwich-...


In [52]:
competitions_df.head(3)

,competition_id,competition_code,name,sub_type,type,country_id,country_name,domestic_league_code,confederation,url
0,CIT,italy-cup,italy-cup,domestic_cup,domestic_cup,75,Italy,IT1,europa,https://www.transfermarkt.co.uk/italy-cup/star...
1,NLSC,johan-cruijff-schaal,johan-cruijff-schaal,domestic_super_cup,other,122,Netherlands,NL1,europa,https://www.transfermarkt.co.uk/johan-cruijff-...
2,GRP,kypello-elladas,kypello-elladas,domestic_cup,domestic_cup,56,Greece,GR1,europa,https://www.transfermarkt.co.uk/kypello-ellada...


In [53]:
competitions_df

,competition_id,competition_code,name,sub_type,type,country_id,country_name,domestic_league_code,confederation,url
0,CIT,italy-cup,italy-cup,domestic_cup,domestic_cup,75,Italy,IT1,europa,https://www.transfermarkt.co.uk/italy-cup/star...
1,NLSC,johan-cruijff-schaal,johan-cruijff-schaal,domestic_super_cup,other,122,Netherlands,NL1,europa,https://www.transfermarkt.co.uk/johan-cruijff-...
2,GRP,kypello-elladas,kypello-elladas,domestic_cup,domestic_cup,56,Greece,GR1,europa,https://www.transfermarkt.co.uk/kypello-ellada...
3,POSU,supertaca-candido-de-oliveira,supertaca-candido-de-oliveira,domestic_super_cup,other,136,Portugal,PO1,europa,https://www.transfermarkt.co.uk/supertaca-cand...
4,RUSS,russian-super-cup,russian-super-cup,domestic_super_cup,other,141,Russia,RU1,europa,https://www.transfermarkt.co.uk/russian-super-...
5,SUC,supercopa,supercopa,domestic_super_cup,other,157,Spain,ES1,europa,https://www.transfermarkt.co.uk/supercopa/star...
6,USC,uefa-super-cup,uefa-super-cup,uefa_super_cup,other,-1,NaN,NaN,europa,https://www.transfermarkt.co.uk/uefa-super-cup...
7,DK1,superligaen,superligaen,first_tier,domestic_league,39,Denmark,DK1,europa,https://www.transfermarkt.co.uk/superligaen/st...
8,EL,europa-league,europa-league,europa_league,international_cup,-1,NaN,NaN,europa,https://www.transfermarkt.co.uk/europa-league/...
9,ES1,laliga,laliga,first_tier,domestic_league,157,Spain,ES1,europa,https://www.transfermarkt.co.uk/laliga/startse...


In [54]:
game_events_df.head(3)

,game_event_id,date,game_id,minute,type,club_id,player_id,description,player_in_id,player_assist_id
0,2f41da30c471492e7d4a984951671677,2012-08-05,2211607,77,Cards,610,4425,"1. Yellow card , Mass confrontation",NaN,NaN
1,a72f7186d132775f234d3e2f7bc0ed5b,2012-08-05,2211607,77,Cards,383,33210,"1. Yellow card , Mass confrontation",NaN,NaN
2,b2d721eaed4692a5c59a92323689ef18,2012-08-05,2211607,3,Goals,383,36500,", Header, 1. Tournament Goal Assist: , Corner,...",NaN,56416.0


In [55]:
game_lineups_df.head(3)

,game_lineups_id,game_id,club_id,type,number,player_id,player_name,team_captain,position
0,b2dbe01c3656b06c8e23e9de714e26bb,2317258,610,substitutes,5,1443,Christian Poulsen,0,Defensive Midfield
1,b50a3ec6d52fd1490aab42042ac4f738,2317258,610,starting_lineup,4,5017,Niklas Moisander,0,Centre-Back
2,7d890e6d0ff8af84b065839966a0ec81,2317258,1090,substitutes,11,9602,Maarten Martens,0,Left Winger


In [56]:
games_df.head(3)

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,...,stadium,attendance,referee,url,home_club_formation,away_club_formation,home_club_name,away_club_name,aggregate,competition_type
0,2321044,L1,2013,2. Matchday,2013-08-18,16,23,2,1,1.0,...,SIGNAL IDUNA PARK,80200.0,Peter Sippel,https://www.transfermarkt.co.uk/borussia-dortm...,4-2-3-1,4-3-2-1,Borussia Dortmund,Eintracht Braunschweig,2:1,domestic_league
1,2321060,L1,2013,3. Matchday,2013-08-25,23,24,0,2,18.0,...,EINTRACHT-Stadion,23325.0,Wolfgang Stark,https://www.transfermarkt.co.uk/eintracht-brau...,4-3-2-1,4-2-3-1,Eintracht Braunschweig,Eintracht Frankfurt Fußball AG,0:2,domestic_league
2,2321086,L1,2013,6. Matchday,2013-09-21,4,16,1,1,15.0,...,Max-Morlock-Stadion,50000.0,Knut Kircher,https://www.transfermarkt.co.uk/1-fc-nuremberg...,4-2-3-1,4-2-3-1,1.FC Nuremberg,Borussia Dortmund,1:1,domestic_league


In [57]:
player_valuations_df.head(3)

,player_id,date,market_value_in_eur,current_club_id,player_club_domestic_competition_id
0,405973,2000-01-20,150000,3057,BE1
1,342216,2001-07-20,100000,1241,SC1
2,3132,2003-12-09,400000,126,TR1


In [58]:
players_df.head(3)

,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
0,10,Miroslav,Klose,Miroslav Klose,2015,398,miroslav-klose,Poland,Opole,Germany,...,right,184.0,NaN,ASBW Sport Marketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,IT1,Società Sportiva Lazio S.p.A.,1000000.0,30000000.0
1,26,Roman,Weidenfeller,Roman Weidenfeller,2017,16,roman-weidenfeller,Germany,Diez,Germany,...,left,190.0,NaN,Neubauer 13 GmbH,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/roman-weidenfe...,L1,Borussia Dortmund,750000.0,8000000.0
2,65,Dimitar,Berbatov,Dimitar Berbatov,2015,1091,dimitar-berbatov,Bulgaria,Blagoevgrad,Bulgaria,...,NaN,NaN,NaN,CSKA-AS-23 Ltd.,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/dimitar-berbat...,GR1,Panthessalonikios Athlitikos Omilos Konstantin...,1000000.0,34500000.0


### `.info()` 확인

In [59]:
# df.info() 확인

dfs = [apps_df, club_games_df, clubs_df, competitions_df, game_events_df, game_lineups_df, 
       games_df, player_valuations_df, players_df]

for df in dfs:
    print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1544716 entries, 0 to 1544715
Data columns (total 13 columns):
 #   Column                  Non-Null Count    Dtype 
---  ------                  --------------    ----- 
 0   appearance_id           1544716 non-null  object
 1   game_id                 1544716 non-null  int64 
 2   player_id               1544716 non-null  int64 
 3   player_club_id          1544716 non-null  int64 
 4   player_current_club_id  1544716 non-null  int64 
 5   date                    1544716 non-null  object
 6   player_name             1544715 non-null  object
 7   competition_id          1544716 non-null  object
 8   yellow_cards            1544716 non-null  int64 
 9   red_cards               1544716 non-null  int64 
 10  goals                   1544716 non-null  int64 
 11  assists                 1544716 non-null  int64 
 12  minutes_played          1544716 non-null  int64 
dtypes: int64(9), object(4)
memory usage: 153.2+ MB
None
<class 'pandas.core.

In [60]:
apps_copy = apps_df.copy()

### 사용 데이터셋
- 목표 : 선수의 시즌별 스탯 확인
    - `apperances` : 선수 출장 기록
    - `competitions` : 대회 정보
    - `players` : 선수에 대한 세부 정보
    
- 해볼 수 있는 것
    - 선수 스탯 예측
    - 선수 몸값 예측

In [61]:
apps_df.describe(include="all")

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
count,1544716,1.544716e+06,1.544716e+06,1.544716e+06,1.544716e+06,1544716,1544715,1544716,1.544716e+06,1.544716e+06,1.544716e+06,1.544716e+06,1.544716e+06
unique,1544716,NaN,NaN,NaN,NaN,3423,23488,43,NaN,NaN,NaN,NaN,NaN
top,2231978_38004,NaN,NaN,NaN,NaN,2020-10-04,Danilo,IT1,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,1795,1065,127706,NaN,NaN,NaN,NaN,NaN
mean,NaN,3.048153e+06,1.861209e+05,2.970430e+03,3.648425e+03,NaN,NaN,NaN,1.486008e-01,3.809114e-03,9.626236e-02,7.580617e-02,6.955760e+01
std,NaN,5.793996e+05,1.709422e+05,7.545136e+03,9.308576e+03,NaN,NaN,NaN,3.670293e-01,6.160039e-02,3.316169e-01,2.862858e-01,2.980235e+01
min,NaN,2.211607e+06,1.000000e+01,1.000000e+00,-1.000000e+00,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,NaN,2.551368e+06,5.534600e+04,2.810000e+02,3.360000e+02,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.100000e+01
50%,NaN,2.985406e+06,1.303600e+05,8.260000e+02,9.310000e+02,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.000000e+01
75%,NaN,3.508190e+06,2.713350e+05,2.441000e+03,2.687000e+03,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.000000e+01


In [62]:
competitions_df.describe(include="all")

,competition_id,competition_code,name,sub_type,type,country_id,country_name,domestic_league_code,confederation,url
count,43,43,43,43,43,43.000000,36,36,43,43
unique,43,42,42,11,4,NaN,14,14,1,43
top,CIT,premier-liga,premier-liga,first_tier,domestic_league,NaN,England,GB1,europa,https://www.transfermarkt.co.uk/italy-cup/star...
freq,1,2,2,14,14,NaN,4,4,43,1
mean,NaN,NaN,NaN,NaN,NaN,97.093023,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,69.766896,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,-1.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,39.500000,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,122.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,157.000000,NaN,NaN,NaN,NaN


In [63]:
players_df.describe()

,player_id,last_season,current_club_id,height_in_cm,market_value_in_eur,highest_market_value_in_eur
count,3.049100e+04,30491.000000,30491.000000,28425.000000,2.934700e+04,2.934700e+04
mean,3.142615e+05,2018.789807,4411.803844,182.250589,1.574263e+06,3.636414e+06
std,2.531756e+05,3.650714,10140.095451,6.839347,6.155242e+06,9.464454e+06
min,1.000000e+01,2012.000000,3.000000,18.000000,1.000000e+04,1.000000e+04
25%,9.597150e+04,2016.000000,403.000000,178.000000,1.000000e+05,2.750000e+05
50%,2.589870e+05,2020.000000,1075.000000,182.000000,2.500000e+05,8.000000e+05
75%,4.703935e+05,2022.000000,3057.000000,187.000000,6.500000e+05,2.800000e+06
max,1.240255e+06,2023.000000,83678.000000,207.000000,1.800000e+08,2.000000e+08


In [64]:
players_df.describe(include="object")

,first_name,last_name,name,player_code,country_of_birth,city_of_birth,country_of_citizenship,date_of_birth,sub_position,position,foot,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name
count,28501,30491,30491,30491,27771,28292,29915,30446,30317,30491,28118,19040,15112,30491,30491,30491,30491
unique,6595,22453,29845,29807,184,8228,182,8932,13,5,3,100,2667,24936,30491,14,424
top,David,García,Paulinho,paulinho,France,London,Spain,1996-01-19,Centre-Back,Defender,right,2024-06-30 00:00:00,Wasserman,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,TR1,Kilmarnock Football Club
freq,202,61,12,12,2185,434,1813,20,5356,9704,19730,5145,435,5556,1,3005,181


#### 사용 데이터셋 병합 함수 생성

In [65]:
# appearance_df, competitions_df, players_df merge 후 19-20 ~ 23-24 시즌 유럽 6대리그 데이터만 남김 (5시즌)
def app_comp_player_merge(apps_copy, competitions_df, players_df):
    # 데이터프레임의 date 컬럼 타입 datetime으로 변경
    apps_copy["date"] = pd.to_datetime(apps_copy["date"])
    
    # apps_copy와 competition_df competition_id를 기준으로 left join
    app_comp_df = pd.merge(apps_copy, competitions_df, how="left", on="competition_id")
    
    # 19-20 ~ 23-24 시즌 데이터만 남김 (5시즌)
    app_comp_df = app_comp_df[app_comp_df["date"]>="2019-06"]

    app_comp_player_df = pd.merge(app_comp_df, players_df, how="left", on="player_id")

#     cols = ['game_id', 'player_id', 'player_club_id',
#             'player_current_club_id', 'date', 'player_name', 'competition_id',
#             'yellow_cards', 'red_cards', 'goals', 'assists', 'minutes_played',
#             'competition_code', 'country_id', 'country_name', 'domestic_league_code', 'last_season',
#             'country_of_citizenship', 'date_of_birth', 'sub_position', 'foot', 'height_in_cm', 
#             'contract_expiration_date', 'current_club_name', 'market_value_in_eur', 'highest_market_value_in_eur']   
     
#     cols2 = ['player_id', 'player_club_id',
#              'player_current_club_id', 'date', 'player_name', 'competition_id',
#              'yellow_cards', 'red_cards', 'goals', 'assists', 'minutes_played',
#              'competition_code', 'country_id', 'country_name', 'domestic_league_code', 'last_season',
#              'country_of_citizenship', 'date_of_birth', 'sub_position', 'foot', 'height_in_cm', 
#              'contract_expiration_date', 'current_club_name', 'market_value_in_eur', 'highest_market_value_in_eur']   

    # 유럽 6대리그 (잉글랜드, 스페인, 이탈리아, 독일, 프랑스, 포르투갈 1부리그)만 필터링
    league_cond = ((app_comp_player_df.competition_id=="GB1") | (app_comp_player_df.competition_id=="ES1") | 
                   (app_comp_player_df.competition_id=="IT1") | (app_comp_player_df.competition_id=="L1") |
                   (app_comp_player_df.competition_id=="FR1") | (app_comp_player_df.competition_id=="PO1"))
#     app_comp_player_df = app_comp_player_df[cols2][league_cond].reset_index(drop=True)
    app_comp_player_df = app_comp_player_df[league_cond].reset_index(drop=True)

    # last_season 제대로 반영되지 않은 것들 발견하여, 새로 변경해준다
    # 8월 (코로나 시즌 7월까지 진행) 을 넘지 않으면 직전 시즌 연도로 변경
    app_comp_player_df["last_season"] = app_comp_player_df["date"].apply(lambda x : x.year if x.month>=8 else x.year-1)

    # 수치형 변수 column 리스트
    group_cols = ["player_name", "last_season", "player_club_id", "yellow_cards", "red_cards","goals", "assists", "minutes_played"]
    
    # groupby를 통해, 이름, 연도 기준으로 수치형 변수 합산 -> 시즌별 기록 파악하기 위함
    df_grouped = app_comp_player_df[group_cols].groupby(["player_name", "last_season", "player_club_id"]).sum()
    df_grouped.reset_index(inplace=True)


    # player_name, last_season, player_club_id을 기준으로 merge 위해, 동일한 연도에 여러 게임으로 나뉘어 나오는 컬럼 제거
    not_merged_cols = group_cols[3:] + ["appearance_id", "game_id", "date", "url_x", "image_url", "url_y"]
    df_not_grouped = app_comp_player_df.loc[:, ~app_comp_player_df.columns.isin(not_merged_cols)].drop_duplicates()

    # 위에서 만든 df_grouped, df_not_grouped merge
    df_merged = pd.merge(df_grouped, df_not_grouped, how="left", on=["player_name", "last_season", "player_club_id"])
    
    # return df_grouped
    return df_merged

In [90]:
# appearance_df, competitions_df, players_df merge 후 19-20 ~ 23-24 시즌 유럽 6대리그 데이터만 남김 (5시즌)
def app_comp_player_merge2(apps_copy, competitions_df, players_df):
    # 데이터프레임의 date 컬럼 타입 datetime으로 변경
    apps_copy["date"] = pd.to_datetime(apps_copy["date"])
    
    # apps_copy와 competition_df competition_id를 기준으로 left join
    app_comp_df = pd.merge(apps_copy, competitions_df, how="left", on="competition_id")
    
    # 19-20 ~ 23-24 시즌 데이터만 남김 (5시즌)
    app_comp_df = app_comp_df[app_comp_df["date"]>="2019-06"]

    app_comp_player_df = pd.merge(app_comp_df, players_df, how="left", on="player_id")

#     cols = ['game_id', 'player_id', 'player_club_id',
#             'player_current_club_id', 'date', 'player_name', 'competition_id',
#             'yellow_cards', 'red_cards', 'goals', 'assists', 'minutes_played',
#             'competition_code', 'country_id', 'country_name', 'domestic_league_code', 'last_season',
#             'country_of_citizenship', 'date_of_birth', 'sub_position', 'foot', 'height_in_cm', 
#             'contract_expiration_date', 'current_club_name', 'market_value_in_eur', 'highest_market_value_in_eur']   
     
#     cols2 = ['player_id', 'player_club_id',
#              'player_current_club_id', 'date', 'player_name', 'competition_id',
#              'yellow_cards', 'red_cards', 'goals', 'assists', 'minutes_played',
#              'competition_code', 'country_id', 'country_name', 'domestic_league_code', 'last_season',
#              'country_of_citizenship', 'date_of_birth', 'sub_position', 'foot', 'height_in_cm', 
#              'contract_expiration_date', 'current_club_name', 'market_value_in_eur', 'highest_market_value_in_eur']   

    # 유럽 6대리그 (잉글랜드, 스페인, 이탈리아, 독일, 프랑스, 포르투갈 1부리그)만 필터링
    league_cond = ((app_comp_player_df.competition_id=="GB1") | (app_comp_player_df.competition_id=="ES1") | 
                   (app_comp_player_df.competition_id=="IT1") | (app_comp_player_df.competition_id=="L1") |
                   (app_comp_player_df.competition_id=="FR1") | (app_comp_player_df.competition_id=="PO1"))
#     app_comp_player_df = app_comp_player_df[cols2][league_cond].reset_index(drop=True)
    app_comp_player_df = app_comp_player_df[league_cond].reset_index(drop=True)

    # last_season 제대로 반영되지 않은 것들 발견하여, 새로 변경해준다
    # 8월 (코로나 시즌 7월까지 진행) 을 넘지 않으면 직전 시즌 연도로 변경
    app_comp_player_df["last_season"] = app_comp_player_df["date"].apply(lambda x : x.year if x.month>=8 else x.year-1)

    # 수치형 변수 column 리스트
    group_cols = ["player_name", "last_season", "player_club_id", "yellow_cards", "red_cards","goals", "assists", "minutes_played"]
    group_cols2 = dict.fromkeys(group_cols, "sum")
    group_cols2["last_season"] = "count"
    
    # groupby를 통해, 이름, 연도 기준으로 수치형 변수 합산 -> 시즌별 기록 파악하기 위함
    # df_grouped = app_comp_player_df[group_cols].groupby(["player_name", "last_season", "player_club_id"]).sum()
    grouped = app_comp_player_df[group_cols].groupby(["player_name", "last_season", "player_club_id"])
    df_grouped = grouped.agg(group_cols2)

    df_grouped.reset_index(inplace=True)


    # player_name, last_season, player_club_id을 기준으로 merge 위해, 동일한 연도에 여러 게임으로 나뉘어 나오는 컬럼 제거
    not_merged_cols = group_cols[3:] + ["appearance_id", "game_id", "date", "url_x", "image_url", "url_y"]
    df_not_grouped = app_comp_player_df.loc[:, ~app_comp_player_df.columns.isin(not_merged_cols)].drop_duplicates()

    # 위에서 만든 df_grouped, df_not_grouped merge
    df_merged = pd.merge(df_grouped, df_not_grouped, how="left", on=["player_name", "last_season", "player_club_id"])
    
    # return df_grouped
    return df_merged

In [86]:
apps_copy.head()

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
0,2231978_38004,2231978,38004,853,235,2012-07-03,Aurélien Joachim,CLQ,0,0,2,0,90
1,2233748_79232,2233748,79232,8841,2698,2012-07-05,Ruslan Abyshov,ELQ,0,0,0,0,90
2,2234413_42792,2234413,42792,6251,465,2012-07-05,Sander Puri,ELQ,0,0,0,0,45
3,2234418_73333,2234418,73333,1274,6646,2012-07-05,Vegar Hedenstad,ELQ,0,0,0,0,90
4,2234421_122011,2234421,122011,195,3008,2012-07-05,Markus Henriksen,ELQ,0,0,0,1,90


In [91]:
app_comp_player_merge2(apps_copy, competitions_df, players_df)

ValueError: cannot insert player_club_id, already exists

In [66]:
df_merged = app_comp_player_merge(apps_copy, competitions_df, players_df)
df_merged.head(3)

,player_name,last_season,player_club_id,yellow_cards,red_cards,goals,assists,minutes_played,player_id,player_current_club_id,...,sub_position,position,foot,height_in_cm,contract_expiration_date,agent_name,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
0,Aaron Connolly,2019,1237,0,0,3,2,1261,434207,1237,...,Centre-Forward,Attack,right,174.0,2024-06-30 00:00:00,NaN,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
1,Aaron Connolly,2020,1237,0,0,2,1,791,434207,1237,...,Centre-Forward,Attack,right,174.0,2024-06-30 00:00:00,NaN,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
2,Aaron Connolly,2021,1237,0,0,0,0,155,434207,1237,...,Centre-Forward,Attack,right,174.0,2024-06-30 00:00:00,NaN,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0


In [67]:
df_merged[df_merged.player_name=="Bryan Gil"]

,player_name,last_season,player_club_id,yellow_cards,red_cards,goals,assists,minutes_played,player_id,player_current_club_id,...,sub_position,position,foot,height_in_cm,contract_expiration_date,agent_name,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
2237,Bryan Gil,2019,368,2,0,0,0,63,537382,148,...,Right Winger,Attack,left,175.0,2026-06-30 00:00:00,IFM,GB1,Tottenham Hotspur Football Club,16000000.0,30000000.0
2238,Bryan Gil,2019,1244,2,0,1,0,553,537382,148,...,Right Winger,Attack,left,175.0,2026-06-30 00:00:00,IFM,GB1,Tottenham Hotspur Football Club,16000000.0,30000000.0
2239,Bryan Gil,2020,368,0,0,0,0,12,537382,148,...,Right Winger,Attack,left,175.0,2026-06-30 00:00:00,IFM,GB1,Tottenham Hotspur Football Club,16000000.0,30000000.0
2240,Bryan Gil,2020,1533,6,0,4,3,2271,537382,148,...,Right Winger,Attack,left,175.0,2026-06-30 00:00:00,IFM,GB1,Tottenham Hotspur Football Club,16000000.0,30000000.0
2241,Bryan Gil,2021,148,0,0,0,0,87,537382,148,...,Right Winger,Attack,left,175.0,2026-06-30 00:00:00,IFM,GB1,Tottenham Hotspur Football Club,16000000.0,30000000.0
2242,Bryan Gil,2021,1049,3,0,0,1,740,537382,148,...,Right Winger,Attack,left,175.0,2026-06-30 00:00:00,IFM,GB1,Tottenham Hotspur Football Club,16000000.0,30000000.0
2243,Bryan Gil,2022,148,1,0,0,1,148,537382,148,...,Right Winger,Attack,left,175.0,2026-06-30 00:00:00,IFM,GB1,Tottenham Hotspur Football Club,16000000.0,30000000.0
2244,Bryan Gil,2022,368,4,0,2,2,922,537382,148,...,Right Winger,Attack,left,175.0,2026-06-30 00:00:00,IFM,GB1,Tottenham Hotspur Football Club,16000000.0,30000000.0
2245,Bryan Gil,2023,148,0,0,0,0,189,537382,148,...,Right Winger,Attack,left,175.0,2026-06-30 00:00:00,IFM,GB1,Tottenham Hotspur Football Club,16000000.0,30000000.0


In [68]:
df_merged.shape

(16267, 38)

In [69]:
df_merged

,player_name,last_season,player_club_id,yellow_cards,red_cards,goals,assists,minutes_played,player_id,player_current_club_id,...,sub_position,position,foot,height_in_cm,contract_expiration_date,agent_name,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
0,Aaron Connolly,2019,1237,0,0,3,2,1261,434207,1237,...,Centre-Forward,Attack,right,174.0,2024-06-30 00:00:00,NaN,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
1,Aaron Connolly,2020,1237,0,0,2,1,791,434207,1237,...,Centre-Forward,Attack,right,174.0,2024-06-30 00:00:00,NaN,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
2,Aaron Connolly,2021,1237,0,0,0,0,155,434207,1237,...,Centre-Forward,Attack,right,174.0,2024-06-30 00:00:00,NaN,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
3,Aaron Cresswell,2019,379,7,0,3,0,2730,92571,379,...,Left-Back,Defender,left,170.0,2024-06-30 00:00:00,Unique Sports Group,GB1,West Ham United Football Club,900000.0,12000000.0
4,Aaron Cresswell,2020,379,3,0,0,8,3172,92571,379,...,Left-Back,Defender,left,170.0,2024-06-30 00:00:00,Unique Sports Group,GB1,West Ham United Football Club,900000.0,12000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16262,İlkay Gündoğan,2019,281,7,0,2,1,2017,53622,131,...,Central Midfield,Midfield,right,180.0,2025-06-30 00:00:00,KIN Partners,ES1,Futbol Club Barcelona,18000000.0,50000000.0
16263,İlkay Gündoğan,2020,281,1,0,13,3,2031,53622,131,...,Central Midfield,Midfield,right,180.0,2025-06-30 00:00:00,KIN Partners,ES1,Futbol Club Barcelona,18000000.0,50000000.0
16264,İlkay Gündoğan,2021,281,1,0,8,3,1855,53622,131,...,Central Midfield,Midfield,right,180.0,2025-06-30 00:00:00,KIN Partners,ES1,Futbol Club Barcelona,18000000.0,50000000.0
16265,İlkay Gündoğan,2022,281,0,0,8,5,2359,53622,131,...,Central Midfield,Midfield,right,180.0,2025-06-30 00:00:00,KIN Partners,ES1,Futbol Club Barcelona,18000000.0,50000000.0


In [70]:
df_merged.to_csv("data/df_merged.csv", index=False)

In [71]:
df_merged.columns

Index(['player_name', 'last_season', 'player_club_id', 'yellow_cards',
       'red_cards', 'goals', 'assists', 'minutes_played', 'player_id',
       'player_current_club_id', 'competition_id', 'competition_code',
       'name_x', 'sub_type', 'type', 'country_id', 'country_name',
       'domestic_league_code', 'confederation', 'first_name', 'last_name',
       'name_y', 'current_club_id', 'player_code', 'country_of_birth',
       'city_of_birth', 'country_of_citizenship', 'date_of_birth',
       'sub_position', 'position', 'foot', 'height_in_cm',
       'contract_expiration_date', 'agent_name',
       'current_club_domestic_competition_id', 'current_club_name',
       'market_value_in_eur', 'highest_market_value_in_eur'],
      dtype='object')

## 결측치 처리

### 결측치 설명
- `완전 무작위 결측 (MCAR : Missing Completely At Random)`

    - 변수 상에서 발생한 결측치가 다른 변수들과 아무런 상관이 없는 경우 완전 무작위 결측이라고 부른다.<br>
    대부분의 결측치 처리 패키지가 MCAR을 가정으로 하고 있고 보통 우리가 생각하는 결측치라고 생각하면 된다.   
    예를 들어, 데이터를 깜박하고 입력 안하거나 전산오류로 누락된 경우이다.   
    이러한 결측치는 보통 제거하거나 데이터 셋에서 단순 무작위 표본추출을 통해 완벽한 데이터셋으로 만들 수 있다.<br>

- `무작위 결측 (MAR : Missing At Random)`

    - 결측값이 결측된 변수와는 관련이 없지만 다른변수와는 관련이 있는 경우 무작위 결측이라고 부른다.  
    예를 들어, 여성이 남성보다 체중을 기입하지 않는다. 라고 하면 체중에 결측값이 생기지만,  
    이는 체중 변수와 관련이 있는 것이 아닌 성별 변수와 관련이 있다. 
<br>

- `비 무작위 결측 (MNAR : Missing Not At Random)`

    - 위 두가지 유형이 아닌 경우 MNAR이라고 하고, 결측값이 결측된 변수와 관련이 있는 경우 비 무작위 결측이라고 한다.  
    예를 들어, 서비스에 불만족한 고객들은 만족도 설문에 응답하지 않는다.  
    
- 링크 : https://syj9700.tistory.com/10

### 결측치 구분 예시
- 구분 예시
    * 3번 상태 (MNAR)가 아니라면, 이 결측값 발생이 어느정도 독립적으로 발생하지 않았을까? 라고 생각 가능한 상태가 되고,
    이는 완벽하게 우연히 발생했다면 1번 완전 무작위 결측이 된다 (MCAR)

    * 즉, 결측값의 발생에 뚜렷한 어떤 인과 관계가 숨어있는지 여부로 구분지으면 된다 
        

    * `MCAR(Missing Completely At Random)`
        
        + 특정 column(c_k)의 결측 여부가 완전히 random으로 발생하는 경우,  

        + 즉 c_k가 다른 c_1, c_2, ...와는 관련이 없을 때를 가리킴


    * `MAR(Missing At Random)  `


        + 특정 c_k의 결측 여부가 다른 c_1, c_2, ... 중 몇몇과 관련이 있는 경우  

        + 예를 들면, 소득수준(=c_1)에 따라 학업성취도(=c_k)의 결측 여부가 관련이 있을 경우
        

    * `MNAR(Missing Not At Random, non ignorable)`  

        + 특정 c_k의 결측 여부가 c_k의 값과 관련이 있는 경우  
        
        + 예를 들면, 학업성취도가 낮은 학생들이 학업성취도에 응답하지 않는 경우   
        
        + 우울증이 너무 심한 응답자들이 우울증 문항에 허위 기술을 하거나,  
        정치적, 종교적인 이유로 실제와 전혀 다른 응답하는 경우  
        
        + 위의 예시처럼 설문지 응답에서 중요한 특정 집단이 의도적으로 자신을 감춰버린 상태

    * 링크 : https://gooopy.tistory.com/43  

### 결측치 비율에 따른 결측치 처리 방법 선택
- 결측치 비율 `10% 미만` : 제거 또는 대체
- 결측치 비율 `10% 이상 20% 미만` : 모델 기반 처리
- 결측치 비율 `20% 이상` : 모델 기반 처리
- 출처 : https://cheris8.github.io/data%20analysis/DP-NA-Imputation/

In [72]:
df_merged.columns

Index(['player_name', 'last_season', 'player_club_id', 'yellow_cards',
       'red_cards', 'goals', 'assists', 'minutes_played', 'player_id',
       'player_current_club_id', 'competition_id', 'competition_code',
       'name_x', 'sub_type', 'type', 'country_id', 'country_name',
       'domestic_league_code', 'confederation', 'first_name', 'last_name',
       'name_y', 'current_club_id', 'player_code', 'country_of_birth',
       'city_of_birth', 'country_of_citizenship', 'date_of_birth',
       'sub_position', 'position', 'foot', 'height_in_cm',
       'contract_expiration_date', 'agent_name',
       'current_club_domestic_competition_id', 'current_club_name',
       'market_value_in_eur', 'highest_market_value_in_eur'],
      dtype='object')

In [73]:
len(df_merged.columns)

38

In [74]:
# 결측치 확인
df_merged.isnull().sum()[:20]

player_name                  0
last_season                  0
player_club_id               0
yellow_cards                 0
red_cards                    0
goals                        0
assists                      0
minutes_played               0
player_id                    0
player_current_club_id       0
competition_id               0
competition_code             0
name_x                       0
sub_type                     0
type                         0
country_id                   0
country_name                 0
domestic_league_code         0
confederation                0
first_name                1617
dtype: int64

In [75]:
df_merged.isnull().sum()[20:]

last_name                                  0
name_y                                     0
current_club_id                            0
player_code                                0
country_of_birth                         522
city_of_birth                             93
country_of_citizenship                   438
date_of_birth                              1
sub_position                               3
position                                   0
foot                                     119
height_in_cm                              55
contract_expiration_date                1477
agent_name                              4506
current_club_domestic_competition_id       0
current_club_name                          0
market_value_in_eur                       30
highest_market_value_in_eur               30
dtype: int64

In [76]:
df_merged[df_merged.first_name.isnull()]["country_of_citizenship"].value_counts()

country_of_citizenship
Brazil                832
Portugal              322
Spain                 157
Cape Verde             24
Angola                 18
Italy                  17
Guinea-Bissau          17
Nigeria                14
Argentina              13
Egypt                  12
Mozambique             11
Morocco                10
South Africa            9
Mexico                  7
France                  6
Ghana                   6
Libya                   6
Togo                    5
Cote d'Ivoire           5
Guinea                  5
United States           5
Denmark                 4
DR Congo                4
Tunisia                 4
Panama                  3
England                 3
Netherlands             3
Japan                   3
Sweden                  2
Colombia                2
Austria                 2
Palestine               2
Cameroon                2
Georgia                 1
Senegal                 1
Kenya                   1
Dominican Republic      1
Algeria        

In [77]:
df_merged["country_of_citizenship"].isnull().sum()

438

In [78]:
df_merged[df_merged.country_of_citizenship.isnull()].iloc[:, :20].head(30)

,player_name,last_season,player_club_id,yellow_cards,red_cards,goals,assists,minutes_played,player_id,player_current_club_id,competition_id,competition_code,name_x,sub_type,type,country_id,country_name,domestic_league_code,confederation,first_name
34,Aaron Zehnter,2022,167,0,0,0,0,1,730484,167,L1,bundesliga,bundesliga,first_tier,domestic_league,40,Germany,L1,europa,Aaron
209,Adil Aouchiche,2019,583,0,0,0,0,65,569385,1158,FR1,ligue-1,ligue-1,first_tier,domestic_league,50,France,FR1,europa,Adil
210,Adil Aouchiche,2020,618,2,0,2,5,1736,569385,1158,FR1,ligue-1,ligue-1,first_tier,domestic_league,50,France,FR1,europa,Adil
211,Adil Aouchiche,2021,618,2,0,0,4,1066,569385,1158,FR1,ligue-1,ligue-1,first_tier,domestic_league,50,France,FR1,europa,Adil
212,Adil Aouchiche,2022,1158,0,0,0,0,241,569385,1158,FR1,ligue-1,ligue-1,first_tier,domestic_league,50,France,FR1,europa,Adil
629,Alex Telles,2019,720,9,1,11,8,2528,255755,368,PO1,liga-portugal-bwin,liga-portugal-bwin,first_tier,domestic_league,136,Portugal,PO1,europa,Alex
630,Alex Telles,2020,720,0,0,2,2,258,255755,368,PO1,liga-portugal-bwin,liga-portugal-bwin,first_tier,domestic_league,136,Portugal,PO1,europa,Alex
631,Alex Telles,2020,985,0,0,0,2,691,255755,368,GB1,premier-league,premier-league,first_tier,domestic_league,189,England,GB1,europa,Alex
632,Alex Telles,2021,985,1,0,0,4,1660,255755,368,GB1,premier-league,premier-league,first_tier,domestic_league,189,England,GB1,europa,Alex
633,Alex Telles,2022,368,5,0,0,2,1426,255755,368,ES1,laliga,laliga,first_tier,domestic_league,157,Spain,ES1,europa,Alex


In [79]:
df_merged[df_merged.country_of_citizenship.isnull()].iloc[:, 20:].head(30)

,last_name,name_y,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,date_of_birth,sub_position,position,foot,height_in_cm,contract_expiration_date,agent_name,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
34,Zehnter,Aaron Zehnter,167,aaron-zehnter,NaN,Sonderhofen,NaN,2004-10-31,Left-Back,Defender,left,180.0,2024-06-30 00:00:00,NaN,L1,FC Augsburg 1907,300000.0,300000.0
209,Aouchiche,Adil Aouchiche,1158,adil-aouchiche,NaN,Le Blanc-Mesnil,NaN,2002-07-15,Attacking Midfield,Midfield,right,181.0,2026-06-30 00:00:00,Sport Cover,FR1,Football Club Lorient-Bretagne Sud,2200000.0,10000000.0
210,Aouchiche,Adil Aouchiche,1158,adil-aouchiche,NaN,Le Blanc-Mesnil,NaN,2002-07-15,Attacking Midfield,Midfield,right,181.0,2026-06-30 00:00:00,Sport Cover,FR1,Football Club Lorient-Bretagne Sud,2200000.0,10000000.0
211,Aouchiche,Adil Aouchiche,1158,adil-aouchiche,NaN,Le Blanc-Mesnil,NaN,2002-07-15,Attacking Midfield,Midfield,right,181.0,2026-06-30 00:00:00,Sport Cover,FR1,Football Club Lorient-Bretagne Sud,2200000.0,10000000.0
212,Aouchiche,Adil Aouchiche,1158,adil-aouchiche,NaN,Le Blanc-Mesnil,NaN,2002-07-15,Attacking Midfield,Midfield,right,181.0,2026-06-30 00:00:00,Sport Cover,FR1,Football Club Lorient-Bretagne Sud,2200000.0,10000000.0
629,Telles,Alex Telles,368,alex-telles,NaN,Caxias do Sul,NaN,1992-12-15,Left-Back,Defender,left,181.0,2024-06-30 00:00:00,Gol International,ES1,Sevilla Fútbol Club S.A.D.,7000000.0,40000000.0
630,Telles,Alex Telles,368,alex-telles,NaN,Caxias do Sul,NaN,1992-12-15,Left-Back,Defender,left,181.0,2024-06-30 00:00:00,Gol International,ES1,Sevilla Fútbol Club S.A.D.,7000000.0,40000000.0
631,Telles,Alex Telles,368,alex-telles,NaN,Caxias do Sul,NaN,1992-12-15,Left-Back,Defender,left,181.0,2024-06-30 00:00:00,Gol International,ES1,Sevilla Fútbol Club S.A.D.,7000000.0,40000000.0
632,Telles,Alex Telles,368,alex-telles,NaN,Caxias do Sul,NaN,1992-12-15,Left-Back,Defender,left,181.0,2024-06-30 00:00:00,Gol International,ES1,Sevilla Fútbol Club S.A.D.,7000000.0,40000000.0
633,Telles,Alex Telles,368,alex-telles,NaN,Caxias do Sul,NaN,1992-12-15,Left-Back,Defender,left,181.0,2024-06-30 00:00:00,Gol International,ES1,Sevilla Fútbol Club S.A.D.,7000000.0,40000000.0


In [80]:
df_merged[(df_merged.first_name.isnull())&(df_merged.country_of_citizenship=="Denmark")]

,player_name,last_season,player_club_id,yellow_cards,red_cards,goals,assists,minutes_played,player_id,player_current_club_id,...,sub_position,position,foot,height_in_cm,contract_expiration_date,agent_name,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
16017,Zanka,2019,38,1,0,0,1,495,52059,1148,...,Centre-Back,Defender,right,189.0,2025-06-30 00:00:00,NaN,GB1,Brentford Football Club,1000000.0,5000000.0
16018,Zanka,2021,1148,2,0,1,0,517,52059,1148,...,Centre-Back,Defender,right,189.0,2025-06-30 00:00:00,NaN,GB1,Brentford Football Club,1000000.0,5000000.0
16019,Zanka,2022,1148,2,0,0,0,1099,52059,1148,...,Centre-Back,Defender,right,189.0,2025-06-30 00:00:00,NaN,GB1,Brentford Football Club,1000000.0,5000000.0
16020,Zanka,2023,1148,2,0,0,0,457,52059,1148,...,Centre-Back,Defender,right,189.0,2025-06-30 00:00:00,NaN,GB1,Brentford Football Club,1000000.0,5000000.0


In [81]:
df_merged[df_merged.player_name=="Adil Aouchiche"]

,player_name,last_season,player_club_id,yellow_cards,red_cards,goals,assists,minutes_played,player_id,player_current_club_id,...,sub_position,position,foot,height_in_cm,contract_expiration_date,agent_name,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
209,Adil Aouchiche,2019,583,0,0,0,0,65,569385,1158,...,Attacking Midfield,Midfield,right,181.0,2026-06-30 00:00:00,Sport Cover,FR1,Football Club Lorient-Bretagne Sud,2200000.0,10000000.0
210,Adil Aouchiche,2020,618,2,0,2,5,1736,569385,1158,...,Attacking Midfield,Midfield,right,181.0,2026-06-30 00:00:00,Sport Cover,FR1,Football Club Lorient-Bretagne Sud,2200000.0,10000000.0
211,Adil Aouchiche,2021,618,2,0,0,4,1066,569385,1158,...,Attacking Midfield,Midfield,right,181.0,2026-06-30 00:00:00,Sport Cover,FR1,Football Club Lorient-Bretagne Sud,2200000.0,10000000.0
212,Adil Aouchiche,2022,1158,0,0,0,0,241,569385,1158,...,Attacking Midfield,Midfield,right,181.0,2026-06-30 00:00:00,Sport Cover,FR1,Football Club Lorient-Bretagne Sud,2200000.0,10000000.0


In [85]:
apps_copy[apps_copy.player_name=="Alex Telles"]

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
215841,2346181_255755,2346181,255755,141,368,2014-02-08,Alex Telles,TR1,0,0,0,0,84
220371,2346192_255755,2346192,255755,141,368,2014-02-17,Alex Telles,TR1,0,0,0,1,90
222334,2346207_255755,2346207,255755,141,368,2014-02-22,Alex Telles,TR1,0,0,0,0,90
224031,2385041_255755,2385041,255755,141,368,2014-02-26,Alex Telles,CL,0,0,0,0,90
226793,2346215_255755,2346215,255755,141,368,2014-03-02,Alex Telles,TR1,0,0,0,0,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1432368,3886677_255755,3886677,255755,368,368,2023-05-14,Alex Telles,ES1,0,0,0,0,67
1435568,3886686_255755,3886686,255755,368,368,2023-05-21,Alex Telles,ES1,0,0,0,0,82
1436126,3886692_255755,3886692,255755,368,368,2023-05-24,Alex Telles,ES1,0,0,0,0,90
1439420,4078394_255755,4078394,255755,368,368,2023-05-31,Alex Telles,EL,0,0,0,0,95
